In [1]:
import numpy as np
import scipy.constants as spc
import pandas as pd
import altair as alt

In [2]:
# Constants
mu_0 = spc.mu_0
pi = spc.pi
mag_const = mu_0/(4*pi)

In [3]:
# Set up survey parameters
D = 100   # Line length in m
h = 2      # Station height in m above ground
N=1000     # Number of points

survey_line=pd.DataFrame({'x':[np.array(xi) for xi in np.vstack([np.linspace(-D,D,N),np.ones((N,))*h]).T]})

In [4]:
# Set up Earth's magnetic field
latitude = 47 # Latitude
I_Earth = np.arctan(2*np.tan(np.radians(latitude)))  # Inclination of Earth's field
B_Earth = 50E-6  # Earth's field magnitude in T (Note: B field, need to divide by mu_0 to get H)

In [18]:
# Set up source parameters

yd = 10.     # Depth of dipole in m
I_dipole = np.radians(80) # Inclination of dipole in degrees (assume N pole is on +X side)
    # Set equal to I_Earth for induced magnetization
mu_magnitude = 1. # Dipole moment magnitude in Am^2

# Calculate dipole moment vector
mu = mu_magnitude*np.array([np.cos(I_dipole), -np.sin(I_dipole)])

def dipole(x_survey, x_dipole, mu):
    r = x_survey-x_dipole
    r_magnitude = np.linalg.norm(r)
    r_hat = r/r_magnitude
    return mag_const*3*((np.dot(mu,r_hat)*r_hat)-mu)/(r_magnitude**3)
    

survey_line['B'] = survey_line['x'].apply(dipole,x_dipole=np.array([0, -yd]), mu=mu)
print(survey_line)

                            x  \
0               [-100.0, 2.0]   
1    [-99.7997997997998, 2.0]   
2    [-99.5995995995996, 2.0]   
3    [-99.3993993993994, 2.0]   
4    [-99.1991991991992, 2.0]   
..                        ...   
995   [99.1991991991992, 2.0]   
996  [99.39939939939941, 2.0]   
997   [99.5995995995996, 2.0]   
998  [99.79979979979981, 2.0]   
999              [100.0, 2.0]   

                                                     B       delta_B  \
0       [3.348442958079718e-14, 2.790369131733098e-13]  1.494621e-19   
1       [3.374764681457307e-14, 2.806673663173953e-13]  1.512479e-19   
2     [3.4013445684554753e-14, 2.8231046426536524e-13]  1.530551e-19   
3      [3.428185651661989e-14, 2.8396632900403374e-13]  1.548841e-19   
4       [3.455291005139338e-14, 2.856350839250154e-13]  1.567352e-19   
..                                                 ...           ...   
995  [-3.6059066973361484e-14, 2.9808588063564587e-13] -7.594800e-19   
996    [-3.577312303565529e

In [19]:
# Calculate anomaly field
survey_line['delta_B']=survey_line['B'].apply(lambda xi: np.dot(xi,np.array([B_Earth*np.cos(np.radians(I_Earth)), 
                                                                             -B_Earth*np.sin(np.radians(I_Earth))])))

In [20]:
# Break vectors x and B into components for plotting
survey_line['transect_distance']=survey_line['x'].apply(lambda xi: xi[0])
survey_line['Bh']=survey_line['B'].apply(lambda xi: xi[0])
survey_line['Bv']=survey_line['B'].apply(lambda xi: xi[1])

In [21]:
# Plot anomaly field
anomaly_plot=alt.Chart(survey_line).mark_line().encode(
    x=alt.X("transect_distance:Q",scale=alt.Scale(domain=(-100,100))),
    y=alt.Y("delta_B:Q")
)
display(anomaly_plot)

alt.Chart(...)